<a href="https://colab.research.google.com/github/nimabahrami/Shapley/blob/main/shapley_value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

the following code is for one of my papers that i wrote in 2022. this could be useful if sb is dealing with group creation for fair allocation of benefits aka **Shapley Value**.

First you need to import the necessary modules.

In [1]:

from itertools import chain, combinations
import math


after importing, the following function creates a list of all possible combinations of the individuals

In [2]:
def get_all_subsets(input_list):
    all_subsets = list(chain.from_iterable(combinations(input_list, r) for r in range(len(input_list)+1)))
    list_of_lists = [set(t) for t in all_subsets]
    return list_of_lists[1::]

In [3]:
print(get_all_subsets(['a','b','c','d']))

[{'a'}, {'b'}, {'c'}, {'d'}, {'a', 'b'}, {'a', 'c'}, {'a', 'd'}, {'b', 'c'}, {'b', 'd'}, {'c', 'd'}, {'a', 'c', 'b'}, {'a', 'b', 'd'}, {'a', 'c', 'd'}, {'b', 'c', 'd'}, {'a', 'c', 'b', 'd'}]


Based on your desired payoff function the payoff dictionary should be created for each subet. for example you have a solar plant e.g. *S*,  and the payoff function shows that if this plant works alone *X* amount will be the payoff *{S}*: *X* . if a wind plant and solar plant mix toghether and generate electricity e.g. S and W, the Y amount will be the payoff *{S,W}*: *Y*

so the following payoff dictionary is just for demonstration.

In [4]:
payoffs = {frozenset({'a'}):0, frozenset({'b'}): 0, frozenset({'c'}): 0, frozenset({'a','c'}):750, frozenset({'a','b'}):750, frozenset({'a','b','c'}):1000, frozenset({'b','c'}):500,}

Next, we have "shapley_value()"

something that is very important is to understand that group creation can happen in multi ways. for example you could have 2 groups of *{S} {W, Es}* but also *{Es, S} {W}* or for more players you could have *{S, W} {Es, Hydro}*  or *{W, Es} {Hydro, S}*

the rest is just mathematical function of shapley

In [5]:
def shapley_value(payoff_mat, input_list):
  subsets = get_all_subsets(input_list)[1::]
  number_of_players = len(input_list)
  marginal_contribution_list = []
  exclusion = []
  result = []
  for i in input_list:
    for j in subsets:
      if not set(i).issubset(j):
        exclusion.append(j)

    for items in exclusion:
      copy_of_items = items.copy()
      copy_of_items.add(i)
      marginal_contribution_val = math.factorial(len(items))*(math.factorial(number_of_players-1-len(items)))*((payoff_mat[frozenset(copy_of_items)])-payoff_mat[frozenset(items)])
      marginal_contribution_list.append(marginal_contribution_val)

    val = (1/math.factorial(number_of_players))*sum(marginal_contribution_list)
    result.append(val)
    exclusion = []
    marginal_contribution_list = []
  return result

In [6]:
print(shapley_value(payoffs,['a','b','c']))

[416.66666666666663, 166.66666666666666, 166.66666666666666]
